In [33]:
from unstructured.documents.elements import Image
import time

In [69]:
import os

#documents_path = '/home/jupyter/project/pdf_data/LIRADS_books_processed/'
#documents_path = '/home/jupyter/project/VP_storage/'
# files = [file for file in os.listdir(documents_path) if '_mainpages.pdf' in file]
# for file in files:
#     print(file)
    
#documents_path = '/home/jupyter/project/pdf_data/LIRADS_papers/' 
documents_path = '/home/jupyter/project/pdf_data/LIRADACR_system_guides/' 
    
files = [file for file in os.listdir(documents_path)]
for file in files:
    print(file)

Application _ RADS - Reporting and Data Systems Support.pdf
LI-RADS CTMR Radiation TRA v2024 Core.pdf
LIRADS Lexicon Table.pdf
Imaging Features _ RADS - Reporting and Data Systems Support.pdf
LI-RADS CTMR Nonradiation TRA v2024 Core.pdf
Getting Started _ RADS - Reporting and Data Systems Support.pdf
Management _ RADS - Reporting and Data Systems Support.pdf
Technique _ RADS - Reporting and Data Systems Support.pdf
Treatment Response _ RADS - Reporting and Data Systems Support.pdf
.ipynb_checkpoints
LI-RADS US Surveillance v2024 Core.pdf
Diagnosis _ RADS - Reporting and Data Systems Support.pdf
Reporting _ RADS - Reporting and Data Systems Support.pdf
Diagnostic Categories _ RADS - Reporting and Data Systems Support.pdf
What's New in v2018 _ RADS - Reporting and Data Systems Support.pdf
LI-RADS 2018 Core.pdf


In [70]:
# path = "/home/jupyter/project/VP_storage/output_books"
# output_folder = "/home/jupyter/project/VP_storage/output_books_tmp"
# tmp_image_path = "/home/jupyter/project/VP_storage/output_books_tmp/tmp_change_name"

# path = "/home/jupyter/project/VP_storage/output_papers"
# output_folder = "/home/jupyter/project/VP_storage/output_papers_tmp"
# tmp_image_path = "/home/jupyter/project/VP_storage/output_books_tmp/tmp_change_name"
# import os
# os.chdir(path)

path = "/home/jupyter/project/VP_storage/output_system_guides"
output_folder = "/home/jupyter/project/VP_storage/output_papers_tmp"
tmp_image_path = "/home/jupyter/project/VP_storage/output_books_tmp/tmp_change_name"
import os
os.chdir(path)

In [71]:

os.environ["EXTRACT_IMAGE_BLOCK_CROP_HORIZONTAL_PAD"] = "20"
#os.environ["EXTRACT_IMAGE_BLOCK_CROP_VERTICAL_PAD "] = "50" #I made silly typo space here, so actually the current doesnt use this setting, nevertheless still a good one

In [135]:
filename = files[15] #to replace

In [131]:
#for books
# folder_name = filename.split('_mainpages.pdf')[0] 
# folder_name = folder_name.split(" ( PDFDrive )")[0] 

# # Define the image path
# image_path = f"./{folder_name}"

# # Create the directory if it doesn't exist
# os.makedirs(image_path, exist_ok=True)

In [132]:
#for non books
folder_name = filename.split('.pdf')[0] 
folder_name = folder_name.split(" ( PDFDrive )")[0] 

# Define the image path
image_path = f"./{folder_name}"

# Create the directory if it doesn't exist
os.makedirs(image_path, exist_ok=True)

In [133]:
import os
import shutil  # To move files from tmp to the final folder
from typing import Any
from pydantic import BaseModel
import time
from PyPDF2 import PdfFileReader, PdfFileWriter
from unstructured.partition.pdf import partition_pdf

class Element(BaseModel):
    type: str
    text: Any
    page_number: int
    table_as_html: Any
    image_path: str
    coordinate : Any

    
# split to temporary files
def split_pdf(input_pdf_path, output_folder, batch_size=50):
    # read raw pdf
    with open(input_pdf_path, 'rb') as input_pdf_file:
        reader = PdfFileReader(input_pdf_file)
        total_pages = reader.numPages
        
        # split to smaller files
        for start_page in range(0, total_pages, batch_size):
            end_page = min(start_page + batch_size, total_pages)
            
            writer = PdfFileWriter()
            for page_num in range(start_page, end_page):
                writer.addPage(reader.getPage(page_num))
            
            # save to tmp files
            output_pdf_path = os.path.join(output_folder, f'tmp_{start_page+1}_to_{end_page}.pdf')
            with open(output_pdf_path, 'wb') as output_pdf_file:
                writer.write(output_pdf_file)
            
            yield output_pdf_path    
    
# Function to categorize elements and adjust the image path
def return_categorized_elements(raw_elements, base_image_path, tmp_image_path, start_page):
    categorized_elements = []
    image_counter = 1

    for element in raw_elements:
        for subelement in element.metadata.orig_elements:
#             if "unstructured.documents.elements.NarrativeText" in str(type(subelement)):
#                 categorized_elements.append(Element(
#                     type="narrative_text",
#                     text=str(subelement),
#                     page_number=subelement.metadata.page_number,
#                     table_as_html='',
#                     image_path=""))
            
#             elif "unstructured.documents.elements.Text" in str(type(subelement)):
#                 categorized_elements.append(Element(
#                     type="text",
#                     text=str(subelement),
#                     page_number=subelement.metadata.page_number,
#                     table_as_html='',
#                     image_path=""))
            
            if "unstructured.documents.elements.Image" in str(type(subelement)):
                # Get the original image path from metadata
                original_image_path = subelement.metadata.image_path
                dir_name, original_filename = os.path.split(original_image_path)

                # Save in the temporary folder (tmp)
                tmp_image_filename = f"tmp_{image_counter}_{original_filename}"
                tmp_image_full_path = os.path.join(tmp_image_path, tmp_image_filename)

                # Save in tmp and adjust as needed
                shutil.copyfile(original_image_path, tmp_image_full_path)  # Copy the image to the tmp folder

                # Modify the file name based on the page number and image sequence
                parts = original_filename.split('-')
                adjusted_page_number = subelement.metadata.page_number + start_page -1
                parts[1] = str(adjusted_page_number)
                new_filename = '-'.join(parts)

                # Create the final path after adjustment
                final_image_full_path = os.path.join(base_image_path, new_filename)

                # Move from tmp folder to the final folder
                shutil.move(tmp_image_full_path, final_image_full_path)

                # Increment image counter for the next image
                image_counter += 1

                # Replace the image_path in the Element object with the adjusted image_full_path
                categorized_elements.append(Element(
                    type="image",
                    text=str(subelement),
                    coordinate = subelement.metadata.coordinates.points,
                    page_number=adjusted_page_number,
                    table_as_html='',
                    
                    image_path=final_image_full_path))  # Updated image_path with new filename

            elif "unstructured.documents.elements.FigureCaption" in str(type(subelement)):
                categorized_elements.append(Element(
                    type="figure_caption",
                    text=str(subelement),
                    page_number=subelement.metadata.page_number,
                    coordinate = subelement.metadata.coordinates.points,
                    table_as_html='',
                    image_path=""))

            # elif "unstructured.documents.elements.ListItem" in str(type(subelement)):
            #     categorized_elements.append(Element(
            #         type="list_item",
            #         text=str(subelement),
            #         page_number=subelement.metadata.page_number,
            #         table_as_html='',
            #         image_path=""))

    # Extract image elements and captions
    image_elements = [e for e in categorized_elements if e.type == "image"]
    captions = [e for e in categorized_elements if e.type == "figure_caption"]

    print(f"Number of images extracted: {len(image_elements)}")
    print(f"Number of captions extracted: {len(captions)}")

    return image_elements, captions, raw_elements


# Function to process temporary PDFs and save images with adjusted names
def process_temp_pdf(pdf_path, image_path, tmp_image_path, start_page):
    # Process PDF using `partition_pdf`
    raw_pdf_elements = partition_pdf(
        filename=pdf_path,
        extract_images_in_pdf=True,
        extract_image_block_types=["Image", "Table"],
        strategy="hi_res",
        infer_table_structure=True,
        chunking_strategy="by_title",
        multipage_sections=True,
        max_characters=4000,
        new_after_n_chars=3800,
        combine_text_under_n_chars=2000,
        extract_image_block_output_dir=tmp_image_path  # Save to the temporary folder (tmp)
    )

    # Send `start_page` to the function to name the images and adjust the image_path correctly
    
    
    return return_categorized_elements(raw_pdf_elements, image_path, tmp_image_path, start_page)


# Main function to process batch pages
def main(documents_path, filename, output_folder, image_path, tmp_image_path, batch_size=100):
    # Step 1: Split the PDF into smaller temporary files
    temp_pdfs = split_pdf(documents_path + filename, output_folder, batch_size)

    # Step 2: Process each temporary file
    #all_text_elements = []
    #all_table_elements = []
    all_image_elements = []
    all_captions = []

    current_start_page = 1  # Start from page 1

    for temp_pdf in temp_pdfs:
        print(f"Processing {temp_pdf}...")

        try:
            # Process the temporary PDF and adjust image names and paths
            image_elements, captions, raw_elements = process_temp_pdf(temp_pdf, image_path, tmp_image_path, current_start_page)
            
            all_text_elements = []
            all_captions = []
            all_raw_elements = []

            # Update start_page for the next batch
            current_start_page += batch_size

            # Append results
            all_image_elements.extend(image_elements)
            all_captions.extend(captions)
            all_raw_elements.extend(raw_elements)

        except OSError as e:
            print(f"Error processing {temp_pdf}: {e}")
            continue

        # Optionally, remove the temporary file after processing
        os.remove(temp_pdf)

    return all_image_elements, all_captions, all_raw_elements

# Example usage
start_time = time.time()

# Define a temporary folder (tmp) to save images before moving them to the final folder
if not os.path.exists(tmp_image_path):
    os.makedirs(tmp_image_path)

# Call the main function
image_elements, captions, raw_elements = main(
    documents_path, filename, output_folder, image_path, tmp_image_path, batch_size=100
)

end_time = time.time()
duration = end_time - start_time
print(f"Processing completed in {duration} seconds.")

# Save results to a file (e.g., using pickle)
import pickle
with open(folder_name+'.pkl', 'wb') as f:
    pickle.dump([image_elements, captions, raw_elements], f)

Processing /home/jupyter/project/VP_storage/output_papers_tmp/tmp_1_to_60.pdf...
Number of images extracted: 64
Number of captions extracted: 2
Processing completed in 231.40668654441833 seconds.


In [113]:
# import os
# os.chdir("/home/jupyter/project/VP_storage/output_books/CT and MRI of the Whole Body, 2-Volume Set, 6e, Volume I/")

## If wanna load the pickle to get the coordinate :

In [ ]:
import pickle

# Define the path to your pickle file
pickle_file_path = "/home/jupyter/project/VP_storage/output_books/gastrointestinal-imaging-the-requisites-fourth-edition.pkl"
# Open the file in binary read mode and load the object
with open(pickle_file_path, 'rb') as file:
    image_elements, captions, raw_elements = pickle.load(file)


In [ ]:
import pandas as pd
image_elements_to_df = [
    {
        'Element_Type': element.type,
        'Text': element.text,
        'Page_Number': element.page_number,
        'Image_Path': element.image_path,
        'Coordinates': element.coordinate
    }
    for element in image_elements
]

# Convert the flattened structure to a DataFrame
df_flattened = pd.DataFrame(image_elements_to_df)

In [18]:
df_flattened

,Element_Type,Text,Page_Number,Image_Path,Coordinates
0,image,A-ring Esophageal vestibule Z line B-ring ...,1,./gastrointestinal-imaging-the-requisites-four...,"((928.35009765625, 462.1430358886719), (928.35..."
1,image,B C D E,2,./gastrointestinal-imaging-the-requisites-four...,"((329.6348571777344, 1575.3785400390625), (329..."
2,image,,3,./gastrointestinal-imaging-the-requisites-four...,"((224.96660027777776, 1307.955508888889), (224..."
3,image,,3,./gastrointestinal-imaging-the-requisites-four...,"((992.966461111111, 1191.6332669444446), (992...."
4,image,B,4,./gastrointestinal-imaging-the-requisites-four...,"((879.4627719444444, 166.6333091666666), (879...."
...,...,...,...,...,...
1290,image,,391,./gastrointestinal-imaging-the-requisites-four...,"((541.6332669444445, 1379.8449197222221), (541..."
1291,image,A,392,./gastrointestinal-imaging-the-requisites-four...,"((305.52215583333333, 26.770951944444555), (30..."
1292,image,A C B D,392,./gastrointestinal-imaging-the-requisites-four...,"((296.62335205078125, 169.48562622070312), (29..."
1293,image,C,392,./gastrointestinal-imaging-the-requisites-four...,"((305.52215583333333, 600.1044802777777), (305..."
